In [ ]:
# 開発過程の補足

# この Jupyter Notebook ファイルは、開発過程を示すために作成されています。  
# セル単位でコードを実行しながら処理内容を確認する「検証用環境」として使用しています。  
# 実際の本番環境では VSCode 上でモジュールを作成し、AWS Lambda および Step Functions によりデータ処理を実行しています。

In [ ]:
# セル1: 共通ベースクラス（BaseScraper）

"""
BaseScraper - スクレイピングの基本機能を提供する基底クラス

このモジュールはスクレイピングの共通機能を持つ基底クラスを提供します。
設定ファイルの読み込み、ブラウザの初期化、ログ機能などの共通機能を集約しています。
"""
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import logging
from dotenv import load_dotenv
from pathlib import Path
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import yaml
import json


class BaseScraper:
    """
    スクレイピングの基本機能を提供する基底クラス
    
    このクラスはスクレイピングに必要な以下の共通機能を提供します:
    - 設定ファイルの読み込み
    - ブラウザの初期化と設定
    - ログ機能
    - CSVファイル操作
    """

    def _load_config(self, config_path=None):
        """
        設定ファイルを読み込みます
        
        Args:
            config_path (str): 設定ファイルのパス
        
        Returns:
            dict: 設定データ
        """
        # デフォルトのパス
        if config_path is None:
            config_path = os.path.join(self.root_dir, 'config', 'settings.yaml')
        
        print(f"設定ファイルパス: {config_path}")
        
        try:
            if not os.path.exists(config_path):
                raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
                
            # YAMLファイルを読み込む
            with open(config_path, 'r', encoding='utf-8') as f:
                config = yaml.safe_load(f)
            
            # 環境変数から認証情報を取得して設定ファイルにマージ
            self._merge_env_variables(config)
            
            print("設定ファイルを正常に読み込みました")
            return config
        except Exception as e:
            print(f"設定ファイルの読み込みエラー: {str(e)}")
            # エラーを上位に伝播させる
            raise

    def load_config(config_path=None):
        """
        設定ファイルを読み込み、環境変数で値を置き換えます
        
        Args:
            config_path (str): 設定ファイルのパス
        
        Returns:
            dict: 設定データ
        """
        # .envファイルを読み込み
        load_dotenv()
        
        # デフォルトのパス
        if config_path is None:
            # プロジェクトルートを特定
            current_dir = os.getcwd()
            project_root = current_dir
            
            # notebooksディレクトリにいる場合はルートに戻る
            if os.path.basename(current_dir) == 'notebooks':
                project_root = os.path.dirname(current_dir)
                
            config_path = os.path.join(project_root, "config", "settings.yaml")
        
        print(f"設定ファイルパス: {config_path}")
        
        try:
            if not os.path.exists(config_path):
                raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
                
            with open(config_path, 'r', encoding='utf-8') as f:
                # YAMLファイルを読み込む
                yaml_content = f.read()
                
                # 環境変数プレースホルダーを置換
                pattern = r'\$\{([A-Za-z0-9_]+)\}'
                
                def replace_env_var(match):
                    env_var = match.group(1)
                    return os.environ.get(env_var, f"${{{env_var}}}")
                
                processed_yaml = re.sub(pattern, replace_env_var, yaml_content)
                
                # 処理済みYAMLを解析
                config = yaml.safe_load(processed_yaml)
            
            print("設定ファイルを正常に読み込みました")
            return config
        except Exception as e:
            print(f"設定ファイルの読み込みエラー: {str(e)}")
            # エラーを上位に伝播させる
            raise

    def _merge_env_variables(self, config):
        """環境変数から認証情報を取得し、設定ファイルにマージする"""
        # このメソッドは子クラスでオーバーライドして実装します
        pass

    def setup_logging(self):
        """ログ機能のセットアップ"""
        # すでに存在するハンドラを削除（重複を防ぐため）
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)
        
        # ログファイルパスの設定
        log_filename = f'{self.site_name}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
        log_file = os.path.join(self.log_dir, log_filename)
        
        # 基本設定
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            encoding='utf-8'
        )
        
        # コンソールにもログを出力
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        console.setFormatter(formatter)
        logging.getLogger('').addHandler(console)
        
        # ログファイルの場所を明示的に表示
        print(f"ログファイル出力先: {log_file}")
        logging.info(f"ログ機能の初期化が完了しました: {log_file}")

    def _find_project_root(self):
        """
        プロジェクトのルートディレクトリを検出する
        """
        # 現在のファイルの絶対パスを取得
        current_dir = os.path.abspath(os.getcwd())
        
        # 親ディレクトリを探索
        path = Path(current_dir)
        while True:
            # .gitディレクトリがあればそれをルートとみなす
            if (path / '.git').exists():
                return str(path)
            
            # プロジェクトのルートを示す他のファイル/ディレクトリの存在チェック
            if (path / 'setup.py').exists() or (path / 'README.md').exists():
                return str(path)
            
            # これ以上上の階層がない場合は現在のディレクトリを返す
            if path.parent == path:
                return str(path)
            
            # 親ディレクトリへ
            path = path.parent

    def _setup_browser(self):
        """
        Seleniumのブラウザを設定します
        
        ChromeOptionsの設定とWebDriverの起動を行います
        """
        # Chromeオプション設定
        chrome_options = Options()
        if self.headless_mode:
            chrome_options.add_argument("--headless")  # ヘッドレスモード（画面表示なし）
        chrome_options.add_argument("--disable-gpu")  # GPU無効化（ヘッドレス時に推奨）
        chrome_options.add_argument("--window-size=1920x1080")  # ウィンドウサイズ設定
        
        try:
            # ドライバーパスが指定されている場合は直接使用
            if hasattr(self, 'driver_path') and self.driver_path:
                self.browser = webdriver.Chrome(service=Service(self.driver_path), options=chrome_options)
            # 指定がなければWebDriverManagerを使用
            else:
                self.browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
            
            self.wait = WebDriverWait(self.browser, 10)  # 要素が見つかるまで最大10秒待機
            print("ブラウザの初期化に成功しました")
        except Exception as e:
            print(f"ブラウザの初期化に失敗しました: {str(e)}")
            raise

    def _setup_session(self):
        """
        requestsのセッションを設定します
        
        Seleniumのブラウザから取得したCookieをrequestsセッションに設定します
        これによりログイン状態を維持したままBeautifulSoupでページ取得できます
        """
        # ブラウザのCookieを取得
        cookies = self.browser.get_cookies()
        
        # requestsセッションを作成
        self.session = requests.Session()
        
        # Seleniumから取得したCookieをrequestsセッションに追加
        for cookie in cookies:
            self.session.cookies.set(cookie['name'], cookie['value'])

    def prepare_csv(self):
        """
        CSVファイルを初期化します
        
        既存のファイルがある場合は削除し、新しいヘッダー付きのCSVを作成します
        """
        # 既存ファイルがあれば削除
        if os.path.exists(self.csv_filename):
            os.remove(self.csv_filename)
        
        # カラム構成でCSVを初期化
        df = pd.DataFrame(columns=self.columns)
        df.to_csv(self.csv_filename, index=False, encoding="utf-8-sig")
        
        print(f"CSVファイル {self.csv_filename} を初期化しました")

    def save_to_csv(self, data):
        """
        データをCSVファイルに保存します
        
        Args:
            data (list): 保存するデータ（リストのリスト形式）
            
        Returns:
            bool: 保存成功時はTrue、失敗時はFalse
        """
        try:
            if data:
                # データフレームに変換
                df = pd.DataFrame(data, columns=self.columns)
                
                # CSVに追記（ヘッダーなし）
                df.to_csv(self.csv_filename, mode="a", index=False, header=False, encoding="utf-8-sig")
                return True
            else:
                print("保存するデータがありません")
                return False
        except Exception as e:
            print(f"CSV保存エラー: {str(e)}")
            return False

    def __init__(self, site_name, config_path=None, headless_mode=False):
        """
        BaseScraperの初期化
        
        Args:
            site_name (str): スクレイピングするサイト名（ログファイル名などに使用）
            config_path (str): 設定ファイルのパス（指定しない場合はデフォルト値を使用）
            headless_mode (bool): ブラウザを画面に表示せずに実行する場合はTrue
        """
        # サイト名の設定
        self.site_name = site_name
        
        # プロジェクトルートディレクトリの検出
        self.root_dir = self._find_project_root()
        
        # 環境変数の読み込み
        load_dotenv(os.path.join(self.root_dir, '.env'))
        
        # ディレクトリパスの設定
        self.data_dir = os.path.join(self.root_dir, 'data')
        self.log_dir = os.path.join(self.root_dir, 'logs')
        
        # ディレクトリが存在しない場合は作成
        os.makedirs(self.data_dir, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        
        # 設定ファイルの読み込み
        self.config = self._load_config(config_path)
        
        # 基本設定
        self.headless_mode = headless_mode
        self.browser = None
        self.wait = None
        self.session = None
        
        # 実行時間計測用
        self.start_time = None
        self.end_time = None

        # ログ設定を初期化時に行う
        self.setup_logging()

    def close(self):
        """
        ブラウザを閉じてリソースを解放します
        """
        if self.browser:
            self.browser.quit()
            print("ブラウザを閉じました")

In [ ]:
# セル2: NETSEA個別クラス（NetseaScraper）

"""
NetseaScraper - ネッシーから商品情報を取得するスクレイピングモジュール

このモジュールはネッシーのウェブサイトから商品情報を取得するための
クラスを提供します。BaseScraper を継承し、NETSEA固有の機能を実装しています。
"""

class NetseaScraper(BaseScraper):
    """
    ネッシーのWebサイトから商品情報をスクレイピングするクラス
    
    このクラスはSeleniumでログインと商品一覧ページの処理を行い、
    BeautifulSoupで個別商品ページの詳細情報を取得するハイブリッド方式を採用しています。
    """
    
    def _merge_env_variables(self, config):
        """環境変数から認証情報を取得し、設定ファイルにマージする"""
        # ネッシーの認証情報
        username = os.getenv('NETSEA_USERNAME')
        password = os.getenv('NETSEA_PASSWORD')
        
        if username and password:
            if 'scrapers' not in config:
                config['scrapers'] = {}
            if 'netsea' not in config['scrapers']:
                config['scrapers']['netsea'] = {}
            if 'login' not in config['scrapers']['netsea']:
                config['scrapers']['netsea']['login'] = {}
            
            config['scrapers']['netsea']['login']['username'] = username
            config['scrapers']['netsea']['login']['password'] = password
            print("ネッシーのログイン情報を環境変数から設定しました")
        else:
            print("警告: 環境変数からネッシーのログイン情報を取得できませんでした")
            
        # デフォルトの出力設定（なければ設定）
        if 'output' not in config['scrapers']['netsea']:
            config['scrapers']['netsea']['output'] = {
                'csv_filename': 'netsea_scraping.csv',
                'log_dir': 'logs'
            }
    
    def __init__(self, config_path=None, headless_mode=False):
        """
        NetseaScraperの初期化
        
        Args:
            config_path (str): 設定ファイルのパス（指定しない場合はデフォルト値を使用）
            headless_mode (bool): ブラウザを画面に表示せずに実行する場合はTrue
        """
        # 親クラス(BaseScraper)の初期化
        super().__init__('netsea', config_path, headless_mode)
        
        # NETSEA固有の設定
        self.netsea_config = self.config['scrapers']['netsea']
        self.base_url = "https://www.netsea.jp"
        
        # 出力設定（設定ファイルから読み込み）
        output_config = self.netsea_config.get('output', {})
        csv_filename = output_config.get('csv_filename', "netsea_scraping.csv")
        
        # CSVのフルパスを設定
        self.csv_filename = os.path.join(self.data_dir, csv_filename)
        print(f"CSVファイル出力先: {self.csv_filename}")
        
        # カラム設定（NETSEA固有）
        self.columns = ["卸業者名", "商品名", "JANコード", "価格", "セット数"]
        
        # ブラウザ設定
        self._setup_browser()
        
        # # ログ設定
        # self.setup_logging()
    
    def login(self, username=None, password=None):
        """
        ネッシーにログインします
        
        Args:
            username (str): ネッシーのログインユーザー名（指定なしの場合は設定ファイルから読み込み）
            password (str): ネッシーのパスワード（指定なしの場合は設定ファイルから読み込み）
            
        Returns:
            bool: ログイン成功時はTrue、失敗時はFalse
        """
        # ユーザー名とパスワードが指定されていない場合は設定ファイルから読み込む
        if username is None or password is None:
            username = self.netsea_config['login']['username']
            password = self.netsea_config['login']['password']
            
        try:
            # ログインページにアクセス
            self.browser.get(f"{self.base_url}/login")
            
            # ログインフォームに入力
            self.wait.until(EC.presence_of_element_located((By.NAME, "login_id"))).send_keys(username)
            self.browser.find_element(By.NAME, "password").send_keys(password)
            self.browser.find_element(By.NAME, "submit").click()
            
            # ログイン成功の確認（ログイン後ページの特定要素が表示されるか）
            self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "globalNav")))
            
            # ログイン後、Cookieを取得してrequestsセッションを準備
            self._setup_session()
            
            return True
        except Exception as e:
            print(f"ログインエラー: {str(e)}")
            return False
    
    def get_product_urls(self, page):
        """
        商品一覧ページから個別商品ページのURLリストを取得します
        
        Args:
            page (int): 取得する商品一覧ページの番号
            
        Returns:
            list: 個別商品ページのURLリスト
        """
        product_urls = []
        print(f"現在 {page} ページ目をスクレイピング中...")
        
        try:
            # 商品一覧ページにアクセス（ここではショップID 5984 を例として使用）
            url = f"{self.base_url}/shop/5984?sort=sales&page={page}"
            self.browser.get(url)
            
            # 商品リストが表示されるまで待機
            self.wait.until(EC.visibility_of_element_located((By.ID, "searchResultsArea")))
            
            # 商品リスト領域を取得
            product_grid = self.browser.find_element(By.ID, "searchResultsArea")
            
            # 各商品要素を取得
            products = product_grid.find_elements(By.CLASS_NAME, "showcaseType01")
            
            # 各商品のURLを取得
            for product in products:
                try:
                    title_block = product.find_element(By.CLASS_NAME, "showcaseHd")
                    product_url = title_block.find_element(By.TAG_NAME, "a").get_attribute("href")
                    product_urls.append(product_url)
                except Exception as e:
                    print(f"商品URL取得エラー（ページ {page}）: {str(e)}")
        
        except Exception as e:
            print(f"ページ {page} の商品リスト取得エラー: {str(e)}")
        
        return product_urls
    
    def get_product_urls_from_url_bs4(self, url, page_number):
        """
        指定されたURLから商品URLリストをBeautifulSoupで取得します
        
        Args:
            url (str): 商品一覧ページのURL
            page_number (int): ページ番号（表示用）
        
        Returns:
            list: 商品URLのリスト
        """
        product_urls = []
        print(f"現在 {page_number} ページ目をスクレイピング中... ({url})")
        
        try:
            # requestsを使ってページを取得
            response = self.session.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 商品リスト領域を探す
            search_results = soup.find(id="searchResultsArea")
            if not search_results:
                print(f"ページ {page_number} の商品リスト領域が見つかりません")
                return product_urls
                
            # 各商品要素を取得
            products = search_results.find_all(class_="showcaseType01")
            
            # 各商品のURLを取得
            for product in products:
                try:
                    title_block = product.find(class_="showcaseHd")
                    if title_block and title_block.a:
                        product_url = title_block.a.get('href')
                        product_urls.append(product_url)
                except Exception as e:
                    print(f"商品URL取得エラー（ページ {page_number}）: {str(e)}")
        
        except Exception as e:
            print(f"ページ {page_number} の商品リスト取得エラー: {str(e)}")
        
        return product_urls
    
    def get_product_data(self, product_urls):
        """
        商品URLリストから詳細データを取得します
        
        Args:
            product_urls (list): 個別商品ページのURLリスト
            
        Returns:
            list: 各商品の詳細データのリスト
        """
        page_data = []
        
        for product_url in product_urls:
            try:
                # requestsとBeautifulSoupを使用してページを取得（高速化のため）
                response = self.session.get(product_url)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 卸業者名（ブランド）を抽出
                brand = "不明"  # デフォルト値
                for script in soup.find_all('script'):
                    if script.string and 'brand' in script.string:
                        brand_match = re.search(r'brand: [\\\'"](.+?)[\\\'"]', script.string)
                        if brand_match:
                            brand = brand_match.group(1)
                            break
                
                # JANコード（gtin13）を抽出 - 修正版
                jan_code = ""  # デフォルト値
                ld_json_scripts = soup.find_all('script', type='application/ld+json')
                for script in ld_json_scripts:
                    if script.string:
                        try:
                            # JSONデータをパース
                            product_data = json.loads(script.string)
                            # Product タイプのJSONで、gtin13が含まれている場合のみ
                            if product_data.get('@type') == 'Product' and 'gtin13' in product_data:
                                jan_code = product_data.get('gtin13', '')
                                break  # 見つかったら終了
                        except json.JSONDecodeError:
                            # JSON解析エラーの場合は正規表現で抽出を試みる
                            jan_match = re.search(r'"gtin13":"(\d+)"', script.string)
                            if jan_match:
                                jan_code = jan_match.group(1)
                                break  # 見つかったら終了
                
                # JANコードが取得できなかった場合、URLから抽出する従来の方法をフォールバックとして使用
                if not jan_code:
                    jan_code = product_url.split('/')[-1]
                    print(f"警告: gtin13が見つからないため、URL末尾をJANコードとして使用: {jan_code}")
                
                # 商品データを含むスクリプトを探す
                script_text = None
                for script in soup.find_all('script'):
                    if script.string and 'ecItemSetList' in script.string:
                        script_text = script.string
                        break
                
                if script_text:
                    # 正規表現で商品情報を抽出（商品名、価格、セット数）
                    items = re.findall(r'_id: .*?_name: [\\\'"](.+?)[\\\'"].*?_priceExcTax: [\\\'"](\d+)[\\\'"].*?_numInSet: [\\\'"](\d+)[\\\'"]', 
                                      script_text, re.DOTALL)
                    
                    # 抽出した情報を整形して追加
                    for name, price, num_in_set in items:
                        row_data = [
                            brand,          # 卸業者名
                            name,           # 商品名
                            jan_code,       # JANコード（新しい方法で取得）
                            price,          # 価格
                            num_in_set      # セット数
                        ]
                        page_data.append(row_data)
                else:
                    print(f"商品データが見つかりませんでした: {product_url}")
                    
            except Exception as e:
                print(f"個別ページ {product_url} のデータ取得エラー: {str(e)}")
        
        return page_data
    
    def scrape_pages(self, start_page=1, end_page=1):
        """
        指定したページ範囲の商品情報をスクレイピングします
        
        Args:
            start_page (int): 開始ページ番号
            end_page (int): 終了ページ番号
            
        Returns:
            int: 取得した商品データの総数
        """
        # 実行時間測定開始
        self.start_time = time.time()
        
        # CSVを初期化
        self.prepare_csv()
        
        total_items = 0
        
        # 指定ページ範囲をスクレイピング
        for page in range(start_page, end_page + 1):
            # 商品URLを取得
            product_urls = self.get_product_urls(page)
            
            # 商品データを取得
            page_data = self.get_product_data(product_urls)
            
            # 1ページごとにCSVに保存
            if page_data:
                self.save_to_csv(page_data)
                total_items += len(page_data)
                print(f"ページ {page} のデータ ({len(page_data)}件) をCSVに保存しました")
        
        # 実行時間測定終了
        self.end_time = time.time()
        elapsed_time = self.end_time - self.start_time
        
        print(f"スクレイピング完了！ 合計{total_items}件のデータを取得しました")
        print(f"実行時間: {elapsed_time:.2f} 秒")
        
        return total_items
    
    def scrape_all_targets(self):
        """
        設定ファイルに指定されたすべてのターゲットページをスクレイピングします
        
        Returns:
            int: 取得した商品データの総数
        """
        # 実行時間測定開始
        self.start_time = time.time()
        
        # CSVを初期化
        self.prepare_csv()
        
        # ログイン（設定ファイルの情報を使用）
        if not self.login():
            print("ログインに失敗しました")
            return 0
        
        # 各ターゲットページの処理
        total_items = 0
        
        for target in self.netsea_config['target_pages']:
            print(f"\n===== {target['name']} の処理を開始 =====")
            
            # URLとページ範囲を取得
            base_url = target['url']
            sort = target.get('sort', '')  # ソート条件（指定がなければ空文字）
            start_page = target.get('start_page', 1)
            end_page = target.get('end_page', 1)
            
            # 各ページの処理
            for page in range(start_page, end_page + 1):
                # URLを構築
                page_url = f"{base_url}?sort={sort}&page={page}"
                
                # 商品URLを取得
                product_urls = self.get_product_urls_from_url_bs4(page_url, page)
                
                # 商品データを取得
                page_data = self.get_product_data(product_urls)
                
                # データを保存
                if page_data:
                    self.save_to_csv(page_data)
                    total_items += len(page_data)
                    print(f"ページ {page} のデータ ({len(page_data)}件) をCSVに保存しました")
        
        # 実行時間測定終了
        self.end_time = time.time()
        elapsed_time = self.end_time - self.start_time
        
        print(f"\n===== スクレイピング完了 - 合計 {total_items} 件 =====")
        print(f"実行時間: {elapsed_time:.2f} 秒")
        
        return total_items

In [ ]:
# スクレイパーのインスタンスを作成して実行するコード
if __name__ == "__main__":
    # スクレイパーのインスタンスを作成
    scraper = NetseaScraper(headless_mode=False)

    try:
        # 設定ファイルに基づいて全ターゲットをスクレイピング
        scraper.scrape_all_targets()
    finally:
        # 終了処理
        scraper.close()

In [ ]:
# # スクレイパーのインスタンスを作成
# scraper = NetseaScraper(headless_mode=False)

# try:
#     # 設定ファイルに基づいて全ターゲットをスクレイピング
#     scraper.scrape_all_targets()
# finally:
#     # 終了処理
#     scraper.close()

In [ ]:
# 413秒/10ページ(628商品) = 0.657秒/1商品　　※ネッシーブレイカーは1.00秒/1商品のため、私のコードのほうが優れている！

In [ ]:
# 401秒/10ページ(628商品) = 0.638秒/1商品
# 5716秒(1.5時間)/166ページ(10250商品) = 0.55秒/1商品　25/3/1

In [ ]:
# メソッドの実行順序
# NetseaScraperを実行すると、以下の順序でメソッドが呼び出されます：

# 初期化フェーズ:

# scraper = NetseaScraper(headless_mode=False) で初期化
# __init__ メソッド内で以下の順序で処理:

# _find_project_root() - プロジェクトルート検出
# 環境変数の読み込み
# ディレクトリパスの設定
# _load_config() - 設定ファイル読み込み

# _merge_env_variables() - 環境変数の設定ファイルへのマージ


# setup_browser() - ブラウザの初期化
# setup_logging() - ログの初期化




# スクレイピング実行フェーズ:

# scraper.scrape_all_targets() の呼び出し
# scrape_all_targets() メソッド内で:

# prepare_csv() - CSV初期化
# login() - ネッシーにログイン

# _setup_session() - リクエストセッションの初期化


# 各ターゲットページに対して:

# get_product_urls_from_url_bs4() - 商品URLの取得
# get_product_data() - 各商品の詳細データ取得
# save_to_csv() - データをCSVに保存






# 終了フェーズ:

# scraper.close() - ブラウザのクローズとリソース解放



# この分割により、共通機能（初期化、設定読み込み、ブラウザ設定など）と固有機能（ネッシー専用のスクレイピングロジック）が明確に分離され